In [1]:
from Users.project.data_container.data_container import AzureStorageAccess
from Users.project.predict_lab_time_module.data_state import DataFeaturing, EFeatureType
from Users.project.predict_lab_time_module.predict_lap_time import PredictLapTime
from Users.project.train_process.model_predictor import MyModelPredictor
from Users.project.train_process.my_utils import extract_track_from_path, timedelta_to_seconds
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
predict_lap_time_process = PredictLapTime(device)
file_name = "2024/2024_United_States_Grand_Prix_R/ALB/car_data_all.csv"
fa = AzureStorageAccess()
df = fa.read_csv_by_data_frame(file_name)
featuring = DataFeaturing()
track_name = extract_track_from_path(file_name)


cuda


In [2]:
rpm = []
speed = []
gear = []
throttle = []
brake = []
drs = []
x_data = []
y_data = []
prev_lap_number = 1.0
prev_lap_time = 0.
for row in df.itertuples():
    rpm.append(row.RPM)
    speed.append(row.Speed)
    gear.append(row.nGear)
    t = row.Throttle
    if t > 100:
        t = 100
    throttle.append(t)
    brake.append(row.Brake)
    drs.append(row.DRS)
    current_lap_number = row.LapNumber
    current_lap_time = row.Time
    if prev_lap_number != current_lap_number:
        # 한 랩이 끝났을 때 저장해둔 모든 데이터로 피쳐만들고 라벨(시간)만들
        feature = []
        label = timedelta_to_seconds(prev_lap_time)
        feature += featuring.feature_by_list(speed, EFeatureType.Basic | EFeatureType.ZeroRatio)                            
        feature += featuring.feature_by_list(rpm, EFeatureType.Basic | EFeatureType.ZeroRatio)                               
        feature += featuring.feature_by_list(gear, EFeatureType.Basic | EFeatureType.Change | EFeatureType.Boolean)       
        feature += featuring.feature_by_list(throttle, EFeatureType.Change | EFeatureType.Boolean)    
        feature += featuring.feature_by_list(brake, EFeatureType.Boolean)                          
        feature += featuring.feature_by_list(drs, EFeatureType.Change | EFeatureType.ZeroRatio | EFeatureType.Boolean)         
        feature += [featuring.trend_feature(speed).item()]    
        feature += [featuring.trend_feature(rpm).item()]       
        feature += [featuring.trend_feature(throttle).item()] 
        rpm.clear(); speed.clear(); gear.clear(); throttle.clear(); brake.clear(); drs.clear()
        x_data.append(feature)
        y_data.append(label)
        break
    prev_lap_number = current_lap_number
    prev_lap_time = current_lap_time

predict_lap_time_process.predict_one_sample(x_data, y_data, track_name)

KeyError: 'United_States_Grand_Prix'

In [3]:
predict_lap_time_process.predict_file(file_name)

TrackNameNotFound:  United_States_Grand_Prix
